In [21]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = glob.glob('../features/*.pkl')
for l in fl:
    print("'{}'".format(l))

'../features/vgg16_1_feat.pkl'
'../features/cnn_1_feat.pkl'
'../features/resnet_aug1_feat.pkl'
'../features/incept_aug1_feat.pkl'
'../features/other_feat.pkl'
'../features/incept_aug2_feat.pkl'
'../features/cnn_1_aug1_feat.pkl'
'../features/resnet_1_feat.pkl'
'../features/cnn_2_aug1_feat_new_preprocess.pkl'
'../features/cnn_2_aug1_feat.pkl'
'../features/cnn_2_feat.pkl'
'../features/cnn_4_aug1_feat.pkl'
'../features/cnn_2_feat_adjust.pkl'
'../features/incept_1_feat.pkl'
'../features/vgg_aug1_feat.pkl'
'../features/resnet_aug2_feat.pkl'
'../features/cnn_2_aug2_feat.pkl'
'../features/cnn_3_aug2_feat.pkl'
'../features/cnn_4_aug2_feat.pkl'


In [22]:
def load_x_y():
    train_x,test_x=[],[]
    for f in fl:
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

(1604, 29) (8424, 29) (1604,)


In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    test_pred = None
    val_loss = 0
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 1.0
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=30,
                        verbose_eval=100)
        
        # cal val loss
        val_loss += log_loss(y_test,m.predict(d_valid))/k_cnt
        
        if test_pred is None:
            test_pred = m.predict(d_test)
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred

    # avg
    test_pred = test_pred / k_cnt
    
    # train log loss
    print('local average valid loss',val_loss)
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/all_xgb_ens_sub.csv', index=False)

    
print('def done')

def done


In [24]:
cv_test(5,233,True)

[0]	train-logloss:0.640042	valid-logloss:0.642126
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.086096	valid-logloss:0.156393
Stopping. Best iteration:
[101]	train-logloss:0.085634	valid-logloss:0.155999

[0]	train-logloss:0.641179	valid-logloss:0.63845
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.096896	valid-logloss:0.099025
[200]	train-logloss:0.050041	valid-logloss:0.087509
[300]	train-logloss:0.029917	valid-logloss:0.084156
Stopping. Best iteration:
[336]	train-logloss:0.025262	valid-logloss:0.082671

[0]	train-logloss:0.63927	valid-logloss:0.643324
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
[100]	train-logloss:0.083739	vali